# **Base de Datos No Relacional**

Para el desarollo del ejercicio, se ha creado **stock.json** como una copia de **BD_NoRelacional_Productos_100.json**.

**a) Descargar el archivo JSON: [BD_NoRelacional_Productos_100.json](https://drive.google.com/file/d/1qUtq8Grcf24yhnfR_l5cDSNckN5XkbGg/view) y responde las siguientes preguntas:**


In [ ]:
import json

def leer_json(ruta_json):
    with open(ruta_json, 'r', encoding='utf-8') as archivo:
        datos = json.load(archivo)
    return datos

no_rel = leer_json("data/BD_NoRelacional_Productos_100.json")

no_rel

- ¿Cómo harías una consulta en MongoDB para obtener los productos de la categoría “smartphones” cuyo precio sea mayor a 2000?

Básicamente recorremos el json, filtrando que la categoría sea **Smarthpones** y el precio sea mayor a **2000**.

In [ ]:
consulta = [p for p in no_rel if p["categoria"] == "Smartphones" and p["precio"] > 2000]

for producto in consulta:
    print(f"{producto['nombre']} - ${producto['precio']}")

- ¿Cómo actualizarías el stock de este producto en MongoDB si se vendieron 3 unidades?

Básicamente podemos agregar un json llamado **ventas**, con el mismo contenido que **stock**, pero con el entry ***stock*** en 0. La idea es reducir la cantidad de ***stock*** en **stock**, y agregarlo en **ventas**.

In [ ]:
stock = leer_json("data/stock.json")
ventas = leer_json("data/ventas.json")

stock
ventas

Podemos reducir el stock, por ejemplo en 3 unidades, definiendo el o los productos que deseamos vender junto con la cantidad. En esta caso lo que deseamos vender lo ponemos en el json **test**.

In [ ]:
test = leer_json("data/test.json")

test

La idea es recorrer el json **stock** de productos a vender y ver si estos hacen match con alguno en stock. Si ello sucede, reducimos el stock, y lo aumentamos en **ventas**. Por si no existe el producto en **ventas**, lo agregamos y seteamos el stock vendido.

In [ ]:
import json

def productos_iguales(prod1, prod2):
    return (
        prod1["nombre"] == prod2["nombre"] and
        prod1["categoria"] == prod2["categoria"] and
        prod1["precio"] == prod2["precio"] and
        prod1["especificaciones"] == prod2["especificaciones"]
    )

def guardar_json(archivo, data):
    with open(archivo, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def actualizar_stock_y_registrar_venta(productos_a_reducir):

    for producto_solicitado in productos_a_reducir:
        nombre = producto_solicitado["nombre"]
        cantidad_a_reducir = producto_solicitado["stock"]

        # Buscar el producto en stock.json con coincidencia exacta
        producto_encontrado = None
        for producto in stock:
            if productos_iguales(producto, producto_solicitado):
                producto_encontrado = producto
                break

        if producto_encontrado:
            if producto_encontrado["stock"] >= cantidad_a_reducir:
                producto_encontrado["stock"] -= cantidad_a_reducir
                print(f"Stock actualizado: {producto_encontrado['stock']} unidades restantes de '{nombre}'")

                # Buscar si el producto ya existe en ventas.json
                producto_en_ventas = None
                for venta in ventas:
                    if productos_iguales(venta, producto_solicitado):
                        producto_en_ventas = venta
                        break

                if producto_en_ventas:
                    producto_en_ventas["stock"] += cantidad_a_reducir
                    print(f"Venta actualizada: {producto_en_ventas['stock']} unidades vendidas de '{nombre}'")
                else:
                    # Si no existe, agregarlo como nueva venta
                    nueva_venta = producto_solicitado.copy()
                    nueva_venta["stock"] = cantidad_a_reducir  # Guardar solo la cantidad vendida
                    ventas.append(nueva_venta)
                    print(f"Venta registrada: {cantidad_a_reducir} unidades de '{nombre}'")

            else:
                print(f"No hay suficiente stock de '{nombre}'. Solo quedan {producto_encontrado['stock']} unidades.")
        else:
            print(f"Producto '{nombre}' con las mismas especificaciones no encontrado en el stock.")

    # Guardar los cambios
    guardar_json("mod_stock.json", stock)
    guardar_json("mod_ventas.json", ventas)

actualizar_stock_y_registrar_venta(test)
stock

- ¿Qué tipo de visualizaciones creerías en QuickSight para monitorear el stock y ventas de productos?

Dado lo anterior, tenemos los json **stock** y **ventas**, entonces para cada uno creamos una visualización, a manera de gráfico de barras, con nombre y stock, y los debidos filtros por si se requiere mayor detalle.

## **Stock**

Podemos ver reflejado el stock de la base de datos no relacional **stock**, con sus debidos filtros.

![Ventas por cliente](figs/fig3.png)

<style>
  img {
    display: block;
    margin: auto;
  }
</style>

## Ventas

Podemos ver reflejado las ventas de la base de datos no relacional **ventas**, con sus debidos filtros.

![Ventas por cliente](figs/fig4.png)

<style>
  img {
    display: block;
    margin: auto;
  }
</style>

- Si los datos en MongoDB cambian dinámicamente, ¿cómo garantizarías que el dashboard de QuickSight siempre tenga datos actualizados?

La idea es mantener la estructura de la imagen. Manejar una función **Lambda** que jale de un **Bucket S3**, lo actualize y haga refresh de las visualizaciones en **Quicksight**.

![Ventas por cliente](figs/fig5.png)

<style>
  img {
    display: block;
    margin: auto;
  }
</style>

Entonces tendremos nuestros json **stock** y **ventas** almacenados en el bucket **jsonstockventas**.

![Ventas por cliente](figs/fig6.png)

<style>
  img {
    display: block;
    margin: auto;
  }
</style>

La siguiente función **Lambda**, basícamente hace lo mismo que antes cuando deseamos actualizar el stock, recibe el json **test** (los productos y la cantidad que se desea vender) que se encuentra más abajo, hace la debida actualización, y hace refresh de los datasets en **Quicksight**, y de esta manera las visualizaciones también se actualizan.

In [ ]:
import json
import boto3
import uuid

s3 = boto3.client('s3')
quicksight = boto3.client('quicksight', region_name='us-east-1')

BUCKET_NAME = "jsonstockventas"
STOCK_FILE = "stock.json"
VENTAS_FILE = "ventas.json"

ACCOUNT_ID = "026123375369"
DATASET_IDS = [
    "55da30ce-cc51-44a5-b557-8d097217181f",
    "0460d905-b5ae-4866-9abd-0fad7b86d002",
    "79e2e1df-6d5d-4606-bf99-a974eb9d2e46"
]

def refresh_quicksight():
    try:
        for dataset_id in DATASET_IDS:
            ingestion_id = f"refresh-{uuid.uuid4()}"
            quicksight.create_ingestion(
                AwsAccountId=ACCOUNT_ID,
                DataSetId=dataset_id,
                IngestionId=ingestion_id
            )
            print(f"QuickSight refresh triggered for {dataset_id}: {ingestion_id}")
        return "QuickSight refresh triggered for all datasets"
    except Exception as e:
        print(f"QuickSight refresh failed: {e}")
        return str(e)

def productos_iguales(prod1, prod2):
    return (
        prod1["nombre"] == prod2["nombre"] and
        prod1["categoria"] == prod2["categoria"] and
        prod1["precio"] == prod2["precio"] and
        prod1["especificaciones"] == prod2["especificaciones"]
    )

def lambda_handler(event, context):
    try:
        if not isinstance(event, list):
            return {"statusCode": 400, "body": "Request must be a list of products"}

        required_fields = ["nombre", "categoria", "precio", "stock", "especificaciones"]
        for product in event:
            for field in required_fields:
                if field not in product:
                    return {"statusCode": 400, "body": f"Missing '{field}' in one of the products"}

        stock_obj = s3.get_object(Bucket=BUCKET_NAME, Key=STOCK_FILE)
        stock_data = json.loads(stock_obj['Body'].read().decode('utf-8'))

        try:
            ventas_obj = s3.get_object(Bucket=BUCKET_NAME, Key=VENTAS_FILE)
            ventas_data = json.loads(ventas_obj['Body'].read().decode('utf-8'))
        except s3.exceptions.NoSuchKey:
            ventas_data = []


        # Buscar el producto en stock.json con coincidencia exacta
        for producto_solicitado in event:
            nombre = producto_solicitado["nombre"]
            cantidad_a_reducir = producto_solicitado["stock"]

            producto_encontrado = None
            for producto in stock_data:
                if productos_iguales(producto, producto_solicitado):
                    producto_encontrado = producto
                    break

            if producto_encontrado:
                if producto_encontrado["stock"] >= cantidad_a_reducir:
                    producto_encontrado["stock"] -= cantidad_a_reducir
                    print(f"Stock actualizado: {producto_encontrado['stock']} unidades restantes de '{nombre}'")


                     # Buscar si el producto ya existe en ventas.json
                    producto_en_ventas = None
                    for venta in ventas_data:
                        if productos_iguales(venta, producto_solicitado):
                            producto_en_ventas = venta
                            break

                    if producto_en_ventas:
                        producto_en_ventas["stock"] += cantidad_a_reducir
                        print(f"Venta actualizada: {producto_en_ventas['stock']} unidades vendidas de '{nombre}'")
                    else:
                        # Si no existe, agregarlo como nueva venta
                        nueva_venta = producto_solicitado.copy()
                        nueva_venta["stock"] = cantidad_a_reducir
                        ventas_data.append(nueva_venta)
                        print(f"Venta registrada: {cantidad_a_reducir} unidades de '{nombre}'")

                else:
                    print(f"No hay suficiente stock de '{nombre}'. Solo quedan {producto_encontrado['stock']} unidades.")
            else:
                print(f"Producto '{nombre}' con las mismas especificaciones no encontrado en el stock.")

        s3.put_object(
            Bucket=BUCKET_NAME,
            Key=STOCK_FILE,
            Body=json.dumps(stock_data, indent=4)
        )

        s3.put_object(
            Bucket=BUCKET_NAME,
            Key=VENTAS_FILE,
            Body=json.dumps(ventas_data, indent=4)
        )

        # Refrescar QuickSight después de la actualización
        refresh_message = refresh_quicksight()

        return {"statusCode": 200, "body": f"Stock actualizado y ventas registradas. {refresh_message}"}

    except Exception as e:
        print(f"Error: {e}")
        return {"statusCode": 500, "body": str(e)}


En el json **test** se pueden agregar tantos productos y cantidad se desee vender.

In [ ]:
test

Entonces con ello se actualizan las visualizaciones de **stock** y **ventas** vistas anteriormente.

# **Stock**

En la visualización **stock** no vemos mucho cambio, porque solo vendemos 3 productos.

![Ventas por cliente](figs/fig7.png)

<style>
  img {
    display: block;
    margin: auto;
  }
</style>

# **Ventas**

En la visualización **ventas** si notamos el cambio, con la venta de estos 3 productos.

![Ventas por cliente](figs/fig8.png)

<style>
  img {
    display: block;
    margin: auto;
  }
</style>

## Scripting

- Crea un script en python llamada: fn_equip_reto_python

- La función debe leer el archivo Excel

- La función debe leer el archivo JSON

- Crea un bucle padre para recorrer cada fila de la base de datos Relacional

- Crea un bucle anidado (dentro del primer bucle) para recorrer cada fila de la base de datos No Relacional

- Cada registro de la base de datos Relacional tendrá un nuevo campo llamado “productos”

- Crea una variable aleatoria X dentro del primer bucle llamado limite_productos el cual variará entre 10 y 50

- Cada campo “producto” se llenará con los X primeros productos de la base de datos No Relacional

Definimos entonces las funciones relevantes, la lectura de **json** y **excel**.

In [6]:
import json
import pandas as pd
import random

def leer_json(ruta_json):
    with open(ruta_json, 'r', encoding='utf-8') as archivo:
        datos = json.load(archivo)
    return datos

def leer_excel(ruta_excel, hoja=0):
    df = pd.read_excel(ruta_excel, sheet_name=hoja, engine='openpyxl')
    return df

Realizamos el bucle anidado indicado, y agregamos la columna **productos**, según la regla indicada.

In [ ]:
#"fn_equip_reto_python.py"

rel = leer_excel("data/BD_Relacional_Ventas_2022_2024.xlsx")
no_rel = leer_json("data/BD_NoRelacional_Productos_100.json")

rel["productos"] = None

for indice, fila in rel.iterrows():
    
    limite_productos = random.randint(10, 50)
    
    productos = []

    for i, producto in enumerate(no_rel):
        if i < limite_productos:
            productos.append(producto)
        else:
            break 

    rel.at[indice, "productos"] = json.dumps(productos, ensure_ascii=False)

rel

**Video con explicación:** [preg2_no_relacional.mp4]()